In [1]:
from sqlalchemy import MetaData, Table, Column
from sqlalchemy import String, Integer

In [2]:
metadata = MetaData()
user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("username", String(50)),
    Column("fullname", String(50)),
)

### slide::
# Then a second table to illustrate multi-table operations
from sqlalchemy import ForeignKey

address_table = Table(
    "email_address",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String(100), nullable=False),
)

In [3]:
from sqlalchemy import create_engine

engine = create_engine("sqlite://")
with engine.begin() as conn:
    metadata.create_all(conn)

In [4]:
# insert data

with engine.begin() as connection:
    connection.execute(
        user_table.insert(),
        [
            {"user_id": 1, "username": "spongebob", "fullname": "Spongebob Squarepants"},
            {"user_id": 2, "username": "sandy", "fullname": "Sandy Cheeks"},
            {"user_id": 3, "username": "patrick", "fullname": "Patrick Star"},
        ],
    )

    connection.execute(
        address_table.insert(),
        [
            {"user_id": 1, "email_address": "spongebob@spongebob.com"},
            {"user_id": 1, "email_address": "spongebob@gmail.com"},
            {"user_id": 2, "email_address": "sandy@yahoo.com"},
            {"user_id": 3, "email_address": "patrick@gmail.com"},
        ],
    )

In [6]:
# title:: SELECT from more than one table, joins
# the select() construct will include in the FROM clause all
# those tables that we mention in the columns clause or WHERE clause.
# by default, they are separated by a comma.
#
from sqlalchemy import select

with engine.connect() as connection:
    stmt = select(user_table.c.username, address_table.c.email_address)
    print(stmt)


SELECT user_account.username, email_address.email_address 
FROM user_account, email_address


In [27]:
# however, selecting from multiple tables without relating them
# to each other produces an effect known as the **cartesian product**.
# SQLAlchemy will usually warn when this is detected during statement
# execution.

# the cartesian result contains every combination of rows which is redundant
# and slow to generate for larger datasets

with engine.connect() as connection:
    stmt = select(user_table.c.username, address_table.c.email_address)
    result = connection.execute(stmt)
    result.all()
    #print(stmt, end='\n\n')
    #print(result.all())
    

In [19]:
# So, when we have more than one table mentioned, we want to relate them
# together, which is most easily achieved using join_from():
with engine.connect() as connection:
    stmt = select(
        user_table.c.username, address_table.c.email_address
    ).join_from(user_table, address_table)

    result = connection.execute(stmt).all()
    print(stmt, end='\n\n')
    print(result)

SELECT user_account.username, email_address.email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id

[('spongebob', 'spongebob@spongebob.com'), ('spongebob', 'spongebob@gmail.com'), ('sandy', 'sandy@yahoo.com'), ('patrick', 'patrick@gmail.com')]


In [18]:
# there is also .join(), which will infer the left hand side automatically

with engine.connect() as connection:
    stmt = select(
        user_table.c.username, address_table.c.email_address
    ).join(address_table)

    result = connection.execute(stmt).all()
    print(stmt, end='\n\n')
    print(result)

SELECT user_account.username, email_address.email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id

[('spongebob', 'spongebob@spongebob.com'), ('spongebob', 'spongebob@gmail.com'), ('sandy', 'sandy@yahoo.com'), ('patrick', 'patrick@gmail.com')]


In [17]:
# the ON clause of the JOIN is also inferred automatically from the
# foreign key relationships of the involved tables.   We may choose
# to express this join condition explicitly, as would be needed if the
# join condition were otherwise ambiguous
with engine.connect() as connection:
    stmt = select(
        user_table.c.username, address_table.c.email_address
    ).join(address_table, user_table.c.id == address_table.c.user_id)

    print(stmt, end='\n\n')
    result = connection.execute(stmt).all()
    print(result)

SELECT user_account.username, email_address.email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id

[('spongebob', 'spongebob@spongebob.com'), ('spongebob', 'spongebob@gmail.com'), ('sandy', 'sandy@yahoo.com'), ('patrick', 'patrick@gmail.com')]


In [15]:
### title:: working with table aliases and subqueries
# When a SELECT wants to refer to the same table more than once, a SQL
# alias is used.  This is available using the  .alias() method, which
# returns a unique Alias object representing that table with a particular
# SQL alias.

with engine.connect() as connection:
    address_alias_1 = address_table.alias()
    address_alias_2 = address_table.alias()

    select_stmt = (
        select(
                user_table.c.username,
                address_alias_1.c.email_address,
                address_alias_2.c.email_address,
        )
        .join_from(user_table, address_alias_1)
        .join_from(user_table, address_alias_2)
        .where(address_alias_1.c.email_address == "spongebob@spongebob.com")
        .where(address_alias_2.c.email_address == "spongebob@gmail.com")
    )
    
    print(select_stmt, end='\n\n')
    result = connection.execute(select_stmt).all()
    print(result)

SELECT user_account.username, email_address_1.email_address, email_address_3.email_address AS email_address_2 
FROM user_account JOIN email_address AS email_address_1 ON user_account.id = email_address_1.user_id JOIN email_address AS email_address_3 ON user_account.id = email_address_3.user_id 
WHERE email_address_1.email_address = :email_address_4 AND email_address_3.email_address = :email_address_5

[('spongebob', 'spongebob@spongebob.com', 'spongebob@gmail.com')]


In [29]:
# A subquery is used much like a table alias, except we start with a select
# statement.   We call the .subquery() method of select()
with engine.connect() as connection:
    select_subq = (
        select(user_table.c.username, address_table.c.email_address)
        .join(address_table).subquery()
    )
    print(select_subq, end='\n\n')
    
    # the subquery object itself has a .c attribute, and is used just like a table.
    stmt = select(select_subq.c.username).where(
    select_subq.c.username == "spongebob"
    )
    print(stmt)

SELECT user_account.username, email_address.email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id

SELECT anon_1.username 
FROM (SELECT user_account.username AS username, email_address.email_address AS email_address 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id) AS anon_1 
WHERE anon_1.username = :username_1


In [31]:
from sqlalchemy import func

address_select = select(
    address_table.c.user_id, func.count(address_table.c.id).label("count")
).group_by(address_table.c.user_id)

address_subq = address_select.subquery()

print(address_subq)

SELECT email_address.user_id, count(email_address.id) AS count 
FROM email_address GROUP BY email_address.user_id


In [32]:
# we use join() to link the subquery() with another select()
with engine.connect() as connection:
    username_plus_count = (
        select(user_table.c.username, address_subq.c.count)
        .join(address_subq)
        .order_by(user_table.c.username)
    )

    result = connection.execute(username_plus_count).all()
    print(result)

[('patrick', 1), ('sandy', 1), ('spongebob', 2)]


In [34]:
### title:: Common Table Expressions
# joining to a subquery can also be achieved using a common table
# expression, or CTE. By calling
# cte() instead of subquery(), we get a CTE:

address_select = select(
    address_table.c.user_id, func.count(address_table.c.id).label("count")
).group_by(address_table.c.user_id)

address_cte = address_select.cte()
print(address_cte)

SELECT email_address.user_id, count(email_address.id) AS count 
FROM email_address GROUP BY email_address.user_id


In [ ]:
# we select/join to the CTE in exactly the same way as we did the subquery.

with engine.connect() as connection:
    username_plus_count = (
        select(user_table.c.username, address_cte.c.count)
        .join(address_cte)
        .order_by(user_table.c.username)
    )

    ### slide:: i

    connection.execute(username_plus_count).all()

In [ ]:
### title:: Correlated subqueries
# a *scalar subquery* returns exactly one row and one column.
# we indicate this intent using the scalar_subquery() method
# after construction

address_corr = (
    select(func.count(address_table.c.id))
    .where(user_table.c.id == address_table.c.user_id)
    .scalar_subquery()
)

# the subquery here refers to two tables.  printing it alone,
# we see both tables in the FROM clause.
print(address_corr)

In [ ]:
# However, a scalar subquery will by default **auto-correlate** in a larger
# SQL expression, omitting a FROM that is found in the immediate
# enclosing SELECT.

select_stmt = select(user_table.c.username, address_corr)
print(select_stmt)